<a href="https://colab.research.google.com/github/Intina47/DEEP_LEARNING/blob/main/malaria/malaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this is a malaria Diagnosis deep learning model